# Part2 Writing a renderer (60 points)
Let’s write a simple software renderer that will transform an object’s 3D coordinates into a 2D picture that is viewed from a camera floating in space.

# Setup

In [1]:
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt

from math import atan, sin, cos
# you are only allowed to import numpy # for further math calculations!! 
import numpy as np

# We import ipywidgets for interactive tools
from ipywidgets import IntSlider, interact, FloatSlider

# Functions

We define 7 functions to capture a object(2D or 3D) in 2D plane as follows:

- show_3d
- show_2d
- get_extrinsic_matrix
- get_intrinsic_matrix
- to_homogenous
- from_homogenous
- take_picture

In [10]:
def show_3d(fig, vertices, edges, **kwargs):
    """
    Show 3D image of a object
    
    Arguments
    ---------
    vertices: vertices of a object
    edges: edges of a object
    fig: a figure object
    
    """
    
    
    # options to plot position and orientation of a camera
    position = kwargs['position'] if 'position' in kwargs.keys() else None
    orientation = kwargs['orientation'] if 'orientation' in kwargs.keys() else None
    
    ax = fig.add_subplot(1,2,1, projection='3d') 
    for edge in edges:
        p1 = vertices[edge[0]] 
        p2 = vertices[edge[1]] 
        ax.plot([p1[0], p2[0]],
                [p1[1], p2[1]],
                [p1[2], p2[2]]) 
        
        # plotting camera position and orientation
        if position is not None:
            p_x, p_y, p_z = position
            ax.scatter(p_x, p_y, p_z, color='red')
            ax.text(p_x+0.1, p_y+0.1, p_z, "camera", color='black')
            
            if orientation is not None:
                o_x, o_y, o_z = orientation
                ax.quiver(p_x, p_y, p_z, o_x/abs(o_x+1e-10), o_y/abs(o_y+1e-10), o_z/abs(o_z+1e-10))
            
    ax.set_ylim(-5, 5)
    ax.set_xlim(-5, 5) 
    ax.set_zlim(-5, 5) 
    ax.set_ylabel('y')
    ax.set_xlabel('x')
    ax.set_zlabel('z')

def show_2d(fig, vertices: list, edges: list, size: list = (1,1)): 
    """
    Show 2D image of a object
    
    Arguments
    ---------
    vertices: vertices of a object (list)
    edges: edges of a object (list)
    size: maximum range of x and y axis (list, default = (1,1))
    fig: a figure object
    
    """
    
    ax = fig.add_subplot(1,2,2)
    for edge in edges:
        p1 = vertices[edge[0]] 
        p2 = vertices[edge[1]] 
        ax.plot([p1[0], p2[0]],
                [p1[1], p2[1]]) 
    ax.set_xlim(0, size[0]) 
    ax.set_ylim(0, size[1])
    ax.set_xlabel('x')
    ax.set_ylabel('y')

def get_extrinsic_matrix(position: list = [0.,0.,-3.], 
                         orientation: list = [0.,0.1,0.], 
                         theta: float = 0.):
    """
    Get a extrinsic matrix to rotate a camera direction
    
    Arguments
    ---------
    position: a camera position in world coordinates (list, default = [0., 0., -3.])
    orientation: a orientation of a camera (list, default = [0., 0.1, 0.])
    theta: a redian for a rotation matrix (float, default = 0.)
    
    Returns
    -------
    R: a rotation matrix
    extrinsic_translate: a matrix multiplication result of extrinsic matrix and translate matrix
    
    """
    
    # define u vector
    orientation = np.array(orientation)
    u = orientation / np.linalg.norm(orientation)
    
    # define cos(theta) and sin(theta)
    c = cos(theta)
    s = sin(theta)
        
    # define R matrix
    R = np.array([[c + (u[0]**2)*(1-c), u[0]*u[1]*(1-c) - u[2]*s, u[0]*u[2]*(1-c) + u[1]*s],
                  [u[1]*u[0]*(1-c) + u[2]*s, c + (u[1]**2)*(1-c), u[1]*u[2]*(1-c) + u[0]*s],
                  [u[2]*u[0]*(1-c) - u[1]*s, u[2]*u[1]*(1-c) + u[0]*s, c + (u[2]**2)*(1-c)]])
    
    
    # rotation matrix
    extrinsic_matrix = np.identity(n=4)
    extrinsic_matrix[:-1, :-1] = R
    
    # translate matrix
    position = np.array(position)
    translate_matrix = np.identity(n=4)
    translate_matrix[:-1,-1] = -np.array(position)
    
    # matrix multiplication of extrinsic matrix and translate matrix
    extrinsic_translate = np.matmul(extrinsic_matrix, translate_matrix)
    
    return R, extrinsic_translate

def to_homogenous(points: list):
    """
    Convert coordinates of a object into homogenous coordinates of a object. The object can be 2D or 3D.
    
    Argument
    --------
    points: coordinates of a object (list)
    
    Return
    ------
    result: homogenous coordinates of a object
    
    """
    
    points = np.array(points)
    result = np.hstack([points, np.ones((points.shape[0], 1))])
    
    return result
        
def from_homogenous(points: list):
    """
    Convert homogenous coordinates of a object into origin coordinates of a object. The object can be 2D or 3D.
    
    Argument
    --------
    points: homogenous coordinates of a object (list)

    Return
    ------
    result: origin coordinates of a object
    
    """
    
    points = np.array(points)
    result = np.array(list(map(lambda x: np.divide(x[:-1], x[-1]), points)))
    
    return result

def get_intrinsic_matrix(f: float = 0.6, s: tuple = (0.5, 0.5)):
    """
    Get intrinsic matrix of a camera
    
    Arguments
    ---------
    f: a focal length from a camera to a object (float, default = 0.6)
    s: principal points (tuple, default = (0.5, 0.5))
    
    Return
    ------
    intrinsic_matrix: a intrinsic matrix of a camera
    
    """
    
    # intrinsic matrix
    intrinsic_matrix = np.array(
        [[f, 0, s[0]],
         [0, f, s[1]],
         [0, 0,   1]]
    )
    
    return intrinsic_matrix

def take_picture(theta: float = 0, 
                 f: float = 0.6, 
                 cx: float = 0., 
                 cy: float = 0., 
                 cz: float = -3., 
                 ox: int = 0, 
                 oy: int = 1, 
                 oz: int = 0):
    """
    Get a image from a object. The object can be 2D or 3D.
    
    Arguments
    ---------
    theta: a radian for a rotation matrix
    f: a focal length from a camera to a object (float, default = 0.6)
    cx: a x-axis of a camera (float, default = 0.) 
    cy: a y-axis of a camera (float, default = 0.)
    cz: a z-axis of a camera (float, default = -3.)
    ox: orientation of x axis (int, default = 0)
    oy: orientation of y axis (int, default = 1)
    oz: orientation of z axis (int, default = 0)
    
    """
    
    # define a cube vertices and edges
    l = 1
    vertices = np.array([[ l, l, l],[ l, l,-l],
                         [ l,-l, l],[-l, l, l],
                         [ l,-l,-l],[-l,-l, l],
                         [-l, l,-l],[-l,-l,-l]])

    edges = []
    for i, v_i in enumerate(vertices):
        for j, v_j in enumerate(vertices):
            if sum(abs(v_i - v_j)) == l*2:
                edges.append([i,j])
    
    # define camera parameters
    position = np.array([cx, cy, cz]) # default = [0,0,-3]
    orientation = np.array([ox, oy, oz]) # default = [0,0.1,0]
    
    # define a focal length and principal points
    f = f
    s = (0.5, 0.5)

    # get extrinsic and intrinsic parameters
    R, extrinsic_translate = get_extrinsic_matrix(position = position, 
                                                  orientation = orientation, 
                                                  theta = theta)
    intrinsic_matrix = get_intrinsic_matrix(f = f, 
                                            s = s)
    
    # make projection matrix
    projection_matrix = np.zeros(shape=(3,4))
    projection_matrix[:,:-1] = np.identity(3)
    
    # parameter result
    param_result = np.matmul(np.matmul(intrinsic_matrix, projection_matrix), extrinsic_translate)
    
    # add homogenous into the cube coordinate
    h_vertices = to_homogenous(points = vertices)
    
    # take a picture
    picture_vertices = np.matmul(param_result, h_vertices.T).T
    picture = from_homogenous(points = picture_vertices)
    
    # show the cube
    rotated_cube = from_homogenous(points = np.matmul(extrinsic_translate, h_vertices.T).T)
    
    
    # make figure
    fig = plt.figure(figsize=(20,10))
    
    rotated_position = np.matmul(R, position.T).T
    
    show_3d(vertices = rotated_cube, 
            edges = edges, 
            fig = fig, 
            position = rotated_position, 
            orientation = orientation-position)
    
    # show the picture
    show_2d(vertices = picture, 
            edges = edges, 
            size = (1,1), 
            fig = fig)
    
    plt.show()

In [13]:
%matplotlib inline
theta_slider = FloatSlider(value=0, min=-1, max=1, step=0.1)
f_slider = FloatSlider(value=0.6, min=0, max=2, step=0.1)
cx_slider = FloatSlider(value=0, min=-6, max=3, step=0.1)
cy_slider = FloatSlider(value=0, min=-6, max=3, step=0.1)
cz_slider = FloatSlider(value=-3, min=-6, max=3, step=0.1)
ox_slider = IntSlider(value=0, min=-1, max=1)
oy_slider = IntSlider(value=1, min=-1, max=1)
oz_slider = IntSlider(value=0, min=-1, max=1)

interact(take_picture, 
         theta=theta_slider,               
         f=f_slider,
         cx=cx_slider,
         cy=cy_slider,
         cz=cz_slider,
         ox=ox_slider,
         oy=oy_slider,
         oz=oz_slider,
)

interactive(children=(FloatSlider(value=0.0, description='theta', max=1.0, min=-1.0), FloatSlider(value=0.6, d…

<function __main__.take_picture(theta: float = 0, f: float = 0.6, cx: float = 0.0, cy: float = 0.0, cz: float = -3.0, ox: int = 0, oy: int = 1, oz: int = 0)>

[TODO]
- orientation을 적용하면 왜 일그러질까

# Bonus (20 points)

Insert code that shows an open tube of similar proportions [you will of course not be able to show the “round” tube with this code – so approximate the round base of the tube with a few lines]!

In [16]:
def draw_3d_polygon(n: int, r: float, h: float):
    """
    Draw a polygon cube
    
    Argument
    --------
    n: the number of vertices (float)
    r: radius (float)
    h: a height of a polygon cube (float)
    
    """
    center_degree = 360 / n
    
    vertices1 = np.array([(r * sin((center_degree * i) * np.pi / 180), h, r * cos((center_degree * i) * np.pi / 180)) for i in range(n)])
    vertices2 = np.array([(r * sin((center_degree * i) * np.pi / 180), -h, r * cos((center_degree * i) * np.pi / 180)) for i in range(n)])

    edges = [[i,i+1] for i in range(n)]
    edges[-1][1] = 0
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.gca(projection='3d') 
    for edge in edges:
        p1 = vertices1[edge[0]] 
        p2 = vertices1[edge[1]]
        
        p3 = vertices2[edge[0]] 
        p4 = vertices2[edge[1]]
        
        ax.plot([p1[0], p2[0]],
                [p1[1], p2[1]],
                [p1[2], p2[2]], color='brown') 
        
        ax.plot([p3[0], p4[0]],
                [p3[1], p4[1]],
                [p3[2], p4[2]], color='brown')
    
        ax.plot([p1[0], p3[0]],
                [p1[1], p3[1]],
                [p1[2], p3[2]], color='brown') 
            
    ax.set_ylim(-5, 5)
    ax.set_xlim(-5, 5) 
    ax.set_zlim(-5, 5) 
    ax.set_ylabel('y')
    ax.set_xlabel('x')
    ax.set_zlabel('z')

In [17]:
%matplotlib inline
r = FloatSlider(value=1, min=1, max=10, step=0.1)
h = FloatSlider(value=1, min=1, max=10, step=0.1)
n = IntSlider(value=3, min=3, max=100, step=1)
interact(draw_3d_polygon, n=n, h=h, r=r)

interactive(children=(IntSlider(value=3, description='n', min=3), FloatSlider(value=1.0, description='r', max=…

<function __main__.draw_3d_polygon(n: int, r: float, h: float)>